In [ ]:
import cv2
import math
import random
import json
from collections import defaultdict
from scipy.spatial import distance
from ultralytics import YOLO  

model = YOLO('../models/first_model.pt')  

video_path = 'http://192.168.69.91:8080/video'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

LABEL_COLORS = {}

def get_label_color(label):
    if label not in LABEL_COLORS:
        LABEL_COLORS[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label]

allowed_pairs = {
    ("tank", "friend"),
    ("enemy", "friend"),
    ("enemy", "wall"),
    ("friend", "wall")
}

centers = []

class ObjectTracker:
    def __init__(self):
        self.objects = {}
        self.next_id = 1
        self.type_counters = defaultdict(int)
    
    def get_unique_name(self, class_name_lower):
        self.type_counters[class_name_lower] += 1
        return f"{class_name_lower.capitalize()}{self.type_counters[class_name_lower]}"
    
    def track_object(self, center_x, center_y, class_name_lower, frame_counter):
        for obj_id, obj_info in self.objects.items():
            if (obj_info['type'] == class_name_lower and 
                distance.euclidean((obj_info['last_x'], obj_info['last_y']), (center_x, center_y)) < 50):
                obj_info.update({
                    'last_x': center_x,
                    'last_y': center_y,
                    'last_seen': frame_counter,
                    'status': 'Active'
                })
                return obj_id
        
        unique_name = self.get_unique_name(class_name_lower)
        new_id = unique_name
        
        self.objects[new_id] = {
            'type': class_name_lower,
            'last_x': center_x,
            'last_y': center_y,
            'last_seen': frame_counter,
            'status': 'Active'
        }
        
        return new_id

    def clean_old_objects(self, frame_counter):
        for obj_id, obj_info in list(self.objects.items()):
            if frame_counter - obj_info['last_seen'] > 10:
                obj_info['status'] = 'Dead'

    def calculate_distances(self, frame_counter):
        distance_data = {}
        active_objects = [
            (obj_id, info['last_x'], info['last_y']) 
            for obj_id, info in self.objects.items() 
            if info['status'] == 'Active'
        ]
        
        for friend_id, friend_x, friend_y in [
            obj for obj in active_objects 
            if self.objects[obj[0]]['type'] == 'friend'
        ]:
            friend_distances = {}
            
            for other_id, other_x, other_y in active_objects:
                if other_id == friend_id:
                    continue
                
                dist_pixels = int(math.dist((friend_x, friend_y), (other_x, other_y)))
                friend_distances[other_id] = f"{dist_pixels}px"
            
            dead_objects = [
                obj_id for obj_id, info in self.objects.items() 
                if info['status'] == 'Dead'
            ]
            
            for dead_id in dead_objects:
                friend_distances[dead_id] = "Dead"
            
            distance_data[friend_id] = friend_distances if friend_distances else "lonely"
        
        return distance_data

object_tracker = ObjectTracker()

frame_counter = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video processing completed.")
        break

    frame_counter += 1
    centers = []

    results = model(frame)
    detections = results[0].boxes  

    for box in detections:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label_id = int(box.cls[0])
            class_name = model.names[label_id].lower()
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            obj_id = object_tracker.track_object(center_x, center_y, class_name, frame_counter)

            color = get_label_color(label_id)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, obj_id, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            centers.append((center_x, center_y, class_name, obj_id))

    for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                p1_x, p1_y, label1, name1 = centers[i]
                p2_x, p2_y, label2, name2 = centers[j]
                if (label1, label2) in allowed_pairs or (label2, label1) in allowed_pairs:
                    distance_pixels = int(math.dist((p1_x, p1_y), (p2_x, p2_y)))
                    cv2.line(frame, (p1_x, p1_y), (p2_x, p2_y), (255, 255, 0), 2)
                    midpoint = ((p1_x + p2_x) // 2, (p1_y + p2_y) // 2)
                    cv2.putText(frame, f"{distance_pixels}px", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)


    object_tracker.clean_old_objects(frame_counter)

    distance_data = object_tracker.calculate_distances(frame_counter)
    print(json.dumps(distance_data, indent=4))

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 Tank, 2 Walls, 1 enemy, 1 friend, 36.0ms
Speed: 1.8ms preprocess, 36.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
{
    "Friend1": {
        "Tank1": "845px",
        "Enemy1": "777px",
        "Wall1": "364px",
        "Wall2": "602px"
    }
}

0: 384x640 2 Tanks, 1 Wall, 1 enemy, 1 friend, 44.4ms
Speed: 1.5ms preprocess, 44.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{
    "Friend1": {
        "Tank1": "841px",
        "Enemy1": "778px",
        "Wall1": "365px",
        "Wall2": "622px",
        "Tank2": "607px"
    }
}

0: 384x640 2 Tanks, 1 Wall, 2 enemys, 1 friend, 44.5ms
Speed: 1.5ms preprocess, 44.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
{
    "Friend1": {
        "Tank1": "842px",
        "Enemy1": "776px",
        "Wall1": "382px",
        "Wall2": "618px",
        "Tank2": "636px",
        "Tank3": "366px",
        "Enemy2": "945px"
    }
}

0: 384x640 1 Tank, 2 Walls, 2 enemys, 2 frie

2025-02-04 15:41:28.684 python[47988:1235031] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-04 15:41:28.684 python[47988:1235031] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 384x640 1 Tank, 2 Walls, 2 enemys, 2 friends, 41.0ms
Speed: 1.4ms preprocess, 41.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
{
    "Friend1": {
        "Tank1": "844px",
        "Enemy1": "782px",
        "Wall1": "467px",
        "Wall2": "672px",
        "Tank2": "752px",
        "Tank3": "447px",
        "Enemy2": "976px",
        "Friend2": "652px",
        "Wall3": "371px"
    },
    "Friend2": {
        "Tank1": "890px",
        "Enemy1": "1022px",
        "Friend1": "652px",
        "Wall1": "701px",
        "Wall2": "397px",
        "Tank2": "432px",
        "Tank3": "712px",
        "Enemy2": "754px",
        "Wall3": "773px"
    }
}

0: 384x640 1 Tank, 2 Walls, 2 enemys, 2 friends, 42.1ms
Speed: 1.3ms preprocess, 42.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
{
    "Friend1": {
        "Tank1": "842px",
        "Enemy1": "779px",
        "Wall1": "475px",
        "Wall2": "674px",
        "Tank2": "760px",
        "Tank3": 

[mjpeg @ 0x16844da20] overread 8


{
    "Friend1": {
        "Tank1": "827px",
        "Enemy1": "767px",
        "Wall2": "712px",
        "Enemy2": "985px",
        "Friend2": "677px",
        "Wall3": "366px",
        "Dead soldier -enemy-2": "974px",
        "Wall9": "475px",
        "Enemy8": "878px",
        "Tank6": "870px",
        "Enemy9": "1036px",
        "Friend6": "662px",
        "Friend7": "104px",
        "Wall10": "764px",
        "Wall1": "Dead",
        "Tank2": "Dead",
        "Tank3": "Dead",
        "Wall4": "Dead",
        "Tank4": "Dead",
        "Enemy3": "Dead",
        "Wall5": "Dead",
        "Friend3": "Dead",
        "Friend4": "Dead",
        "Enemy4": "Dead",
        "Wall6": "Dead",
        "Enemy5": "Dead",
        "Friend5": "Dead",
        "Tank5": "Dead",
        "Wall7": "Dead",
        "Wall8": "Dead",
        "Enemy6": "Dead",
        "Enemy7": "Dead",
        "Dead soldier -enemy-1": "Dead"
    },
    "Friend2": {
        "Tank1": "877px",
        "Enemy1": "1013px",
        "F

In [1]:
# JSON Correct. Object name not
import cv2
import math
import random
import json
from collections import defaultdict
from scipy.spatial import distance
from ultralytics import YOLO  

# Load the YOLO model
model = YOLO('../models/first_model.pt')  # Change to your model path

# Open the video file
video_path = '../test_vids/battlefield-1.mp4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

# Store previous object locations for tracking
object_tracker = {}

# Maximum distance to consider an object as the same across frames
MAX_TRACKING_DISTANCE = 50

# Frame counter to remove old objects
frame_counter = 0
MAX_FRAMES_MISSING = 10  # If an object disappears for this many frames, we remove it

def calculate_distances(friends, other_objects):
    """
    Calculate distances between friends and all other objects
    
    Args:
    friends (list): List of (name, x, y) for friend objects
    other_objects (list): List of (name, x, y) for other objects
    
    Returns:
    dict: Distance data between friends and other objects
    """
    distance_data = {}
    
    # For each friend
    for friend_name, friend_x, friend_y in friends:
        friend_distances = {}
        
        # Calculate distance to each other object
        for obj_name, obj_x, obj_y in other_objects:
            # Avoid calculating distance to self
            if friend_name != obj_name:
                dist_pixels = int(math.dist((friend_x, friend_y), (obj_x, obj_y)))
                friend_distances[obj_name] = f"{dist_pixels}px"
        
        # If no other objects, mark friend lonely
        if not friend_distances:
            friend_distances = "lonely"
        
        distance_data[friend_name] = friend_distances
    
    return distance_data

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video processing completed.")
        break

    frame_counter += 1

    # Perform object detection
    results = model(frame)
    detections = results[0].boxes  

    friends = []  # Track friends
    other_objects = []  # Track other objects

    # Draw bounding boxes, labels, and store centers
    for box in detections:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Extract box coordinates
        confidence = box.conf[0]  # Extract confidence score
        label_id = int(box.cls[0])  # Extract class label ID

        # Get the class name from the model
        class_name = model.names[label_id]  
        class_name_lower = class_name.lower()

        # Calculate the center point
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Try to match with existing tracked objects
        assigned_id = None
        for obj_id, (prev_x, prev_y, prev_label, last_seen) in object_tracker.items():
            if prev_label == class_name_lower and distance.euclidean((prev_x, prev_y), (center_x, center_y)) < MAX_TRACKING_DISTANCE:
                assigned_id = obj_id
                break

        # If no match found, assign a new ID
        if assigned_id is None:
            assigned_id = len(object_tracker) + 1

        # Create unique name
        unique_name = f"{class_name} {assigned_id}"

        # Store object for the next frame
        object_tracker[assigned_id] = (center_x, center_y, class_name_lower, frame_counter)

        # Separate friends from other objects
        if class_name_lower == 'friend':
            friends.append((unique_name, center_x, center_y))
        else:
            other_objects.append((unique_name, center_x, center_y))

    # Remove old objects that haven't been seen for a while
    object_tracker = {obj_id: data for obj_id, data in object_tracker.items() 
                      if frame_counter - data[3] < MAX_FRAMES_MISSING}

    # Calculate distances if friends exist
    if friends:
        distance_data = calculate_distances(friends, other_objects)
        
        # Print the current distance data
        print(json.dumps(distance_data, indent=4))

    # Display the frame (optional, can be removed)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()
cv2.destroyAllWindows()


0: 640x384 1 Dead soldier -frd-, 1 Tank, 2 Walls, 2 enemys, 1 friend, 40.7ms
Speed: 2.8ms preprocess, 40.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)
{
    "friend 4": {
        "Wall 1": "193px",
        "Tank 2": "367px",
        "Wall 3": "228px",
        "enemy 5": "291px",
        "Dead soldier -frd- 6": "352px",
        "enemy 7": "315px"
    }
}

0: 640x384 1 Dead soldier -frd-, 1 Tank, 2 Walls, 2 enemys, 1 friend, 41.5ms
Speed: 1.2ms preprocess, 41.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
{
    "friend 4": {
        "Wall 1": "193px",
        "Tank 2": "368px",
        "Wall 3": "228px",
        "enemy 5": "292px",
        "Dead soldier -frd- 6": "354px",
        "enemy 7": "315px"
    }
}

0: 640x384 1 Dead soldier -frd-, 1 Tank, 2 Walls, 2 enemys, 1 friend, 53.0ms
Speed: 1.3ms preprocess, 53.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
{
    "friend 4": {
        "Wall 1": "193px",
        "Tank 2": 

2025-02-04 02:22:14.599 python[27501:543975] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-04 02:22:14.599 python[27501:543975] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 640x384 2 Dead soldier -frd-s, 1 Tank, 2 Walls, 2 enemys, 1 friend, 49.7ms
Speed: 1.2ms preprocess, 49.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
{
    "friend 4": {
        "Tank 2": "369px",
        "Wall 1": "194px",
        "Wall 3": "228px",
        "enemy 5": "292px",
        "Dead soldier -frd- 6": "345px",
        "enemy 7": "316px"
    }
}

0: 640x384 2 Dead soldier -frd-s, 1 Tank, 2 Walls, 2 enemys, 1 friend, 41.1ms
Speed: 1.1ms preprocess, 41.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)
{
    "friend 4": {
        "Wall 1": "195px",
        "Tank 2": "369px",
        "Wall 3": "230px",
        "enemy 5": "294px",
        "Dead soldier -frd- 6": "343px",
        "enemy 7": "318px"
    }
}

0: 640x384 1 Dead soldier -frd-, 1 Tank, 2 Walls, 2 enemys, 1 friend, 43.4ms
Speed: 1.1ms preprocess, 43.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
{
    "friend 4": {
        "Wall 1": "195px",
        "Tank 2":

In [ ]:
import cv2
import math
import random
import json
from collections import defaultdict
from scipy.spatial import distance
from ultralytics import YOLO  

# Load the YOLO model
model = YOLO('../models/first_model.pt')  # Change to your model path

# Open the video file
video_path = '../test_vids/battlefield-1.mp4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

# Define a dictionary to store unique colors for each label
LABEL_COLORS = {}

# Function to generate random colors for labels
def get_label_color(label_id):
    if label_id not in LABEL_COLORS:
        LABEL_COLORS[label_id] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label_id]

# Define the allowed object pairs for distance calculation
allowed_pairs = {
    ("tank", "friend"),
    ("enemy", "friend"),
    ("enemy", "wall"),
    ("friend", "wall")
}

# Store previous object locations for tracking
object_tracker = {}

# Maximum distance to consider an object as the same across frames
MAX_TRACKING_DISTANCE = 50

# Frame counter to remove old objects
frame_counter = 0
MAX_FRAMES_MISSING = 10  # If an object disappears for this many frames, we remove it

# Dictionary to store distance data
distance_data = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video processing completed.")
        break

    frame_counter += 1

    # Perform object detection
    results = model(frame)
    detections = results[0].boxes  

    new_objects = []  # Store new detections in this frame

    # Draw bounding boxes, labels, and store centers
    for box in detections:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Extract box coordinates
        confidence = box.conf[0]  # Extract confidence score
        label_id = int(box.cls[0])  # Extract class label ID

        # Get the class name from the model
        class_name = model.names[label_id]  
        class_name_lower = class_name.lower()

        # Calculate the center point
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Try to match with existing objects
        assigned_id = None
        for obj_id, (prev_x, prev_y, prev_label, last_seen) in object_tracker.items():
            if prev_label == class_name_lower and distance.euclidean((prev_x, prev_y), (center_x, center_y)) < MAX_TRACKING_DISTANCE:
                assigned_id = obj_id
                break

        # If no match found, assign a new ID
        if assigned_id is None:
            assigned_id = len(object_tracker) + 1

        # Store object for the next frame
        object_tracker[assigned_id] = (center_x, center_y, class_name_lower, frame_counter)
        unique_name = f"{class_name} {assigned_id}"  # E.g., "Friend 1", "Tank 2"

        # Debugging: Print detected object names
        print(f"Detected: {unique_name}")

        # Assign a unique color to this label
        color = get_label_color(label_id)

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Draw the label and confidence score
        text = f'{unique_name} {confidence:.2f}'
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Draw the center point with the same label color
        cv2.circle(frame, (center_x, center_y), 5, color, -1)

        # Store for distance calculation
        new_objects.append((center_x, center_y, class_name_lower, unique_name))

    # Remove old objects that haven't been seen for a while (consider them "dead")
    object_tracker = {obj_id: data for obj_id, data in object_tracker.items() if frame_counter - data[3] < MAX_FRAMES_MISSING}

    # Calculate distances only for the allowed object pairs
    for i in range(len(new_objects)):
        for j in range(i + 1, len(new_objects)):
            p1_x, p1_y, label1, name1 = new_objects[i]
            p2_x, p2_y, label2, name2 = new_objects[j]

            # Check if the pair is in the allowed set (both converted to lowercase)
            if (label1, label2) in allowed_pairs or (label2, label1) in allowed_pairs:
                distance_pixels = int(math.dist((p1_x, p1_y), (p2_x, p2_y)))

                # Add the distance to the JSON-like dictionary, only if the objects are "alive"
                if label1 != label2:  # To avoid self-distances
                    if name1 not in distance_data:
                        distance_data[name1] = {}
                    distance_data[name1][name2] = f"{distance_pixels}px"

                # Draw line between allowed pairs
                cv2.line(frame, (p1_x, p1_y), (p2_x, p2_y), (255, 255, 0), 2)  # Yellow line

                # Display distance at the midpoint of the line
                midpoint = ((p1_x + p2_x) // 2, (p1_y + p2_y) // 2)
                cv2.putText(frame, f"{distance_pixels}px", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Print the current distance data (for debugging purposes)
    print(json.dumps(distance_data, indent=4))

    # Display the frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()
cv2.destroyAllWindows()


In [ ]:
#only with distance between labels
import cv2
import math
import random
from ultralytics import YOLO  

# Load the YOLO model
model = YOLO('../models/first_model.pt')  # Change to your model path

# Open the video file
video_path = '../test_vids/battlefield-1.mp4'
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

# Define a dictionary to store unique colors for each label
LABEL_COLORS = {}

# Function to generate random colors for labels
def get_label_color(label_id):
    if label_id not in LABEL_COLORS:
        LABEL_COLORS[label_id] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label_id]

# Define the allowed object pairs for distance calculation
allowed_pairs = {
    ("tank", "friend"),
    ("enemy", "friend"),
    ("enemy", "wall"),
    ("friend", "wall")
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video processing completed.")
        break

    # Perform object detection
    results = model(frame)
    detections = results[0].boxes  

    centers = []  # Store center points of detected objects along with their labels

    # Draw bounding boxes, labels, and store centers
    for box in detections:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Extract box coordinates
        confidence = box.conf[0]  # Extract confidence score
        label_id = int(box.cls[0])  # Extract class label ID

        # Get the class name from the model
        class_name = model.names[label_id]  # Keep original detected name
        class_name_lower = class_name.lower()  # Convert to lowercase for matching

        # Debugging: Print detected object names
        print(f"Detected: {class_name}")

        # Assign a unique color to this label
        color = get_label_color(label_id)

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # Draw the label and confidence score
        text = f'{class_name} {confidence:.2f}'
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Calculate the center point
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        centers.append((center_x, center_y, class_name_lower))  # Store lowercase name

        # Draw the center point with the same label color
        cv2.circle(frame, (center_x, center_y), 5, color, -1)

    # Calculate distances only for the allowed object pairs
    for i in range(len(centers)):
        for j in range(i + 1, len(centers)):
            p1_x, p1_y, label1 = centers[i]
            p2_x, p2_y, label2 = centers[j]

            # Check if the pair is in the allowed set (both converted to lowercase)
            if (label1, label2) in allowed_pairs or (label2, label1) in allowed_pairs:
                distance_pixels = int(math.dist((p1_x, p1_y), (p2_x, p2_y)))

                # Draw line between allowed pairs
                cv2.line(frame, (p1_x, p1_y), (p2_x, p2_y), (255, 255, 0), 2)  # Yellow line

                # Display distance at the midpoint of the line
                midpoint = ((p1_x + p2_x) // 2, (p1_y + p2_y) // 2)
                cv2.putText(frame, f"{distance_pixels}px", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Display the frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()
cv2.destroyAllWindows()
